# 03_semantic_search_demo.ipynb — Semantic search (SBERT + OpenAI)

Цей ноутбук: побудова семантичного пошуку поверх унікальних StackOverflow title’ів.

- Sentence Transformers embeddings + NearestNeighbors (cosine)
- OpenAI embeddings (`text-embedding-3-large`) + NearestNeighbors (cosine)
- Демо запитів і якісне порівняння


## Installs (Colab)

In [ ]:
!pip install -q datasets scikit-learn pandas matplotlib sentence-transformers openai

## Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer


## Load dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "sentence-transformers/stackexchange-duplicates",
    "title-title-pair"
)

# Convert to pandas for convenience
df = dataset["train"].to_pandas()
df.head()

###Semantic Search модуль

In [ ]:
all_titles = pd.concat([df["title1"], df["title2"]], ignore_index=True)

# Унікальні заголовки
unique_titles = all_titles.drop_duplicates().reset_index(drop=True)
len(unique_titles)

In [ ]:
unique_titles.head()

Завантажуємо SentenceTransformer і рахуємо ембедінги

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)

In [ ]:
# Ембедінги для всіх унікальних заголовків
title_embeddings = model.encode(
    unique_titles.tolist(),
    batch_size=64,
    convert_to_numpy=True,
    show_progress_bar=True,
    device=device
)

title_embeddings = title_embeddings.astype("float32")
title_embeddings.shape

NearestNeighbors з cosine metric

In [ ]:
# Створюємо brute-force індекс з cosine-відстанню
nn = NearestNeighbors(
    n_neighbors=5,
    metric="cosine"   # 1 - cos_sim
)

nn.fit(title_embeddings)

Функція semantic search

In [ ]:
def semantic_search(query, model, nn, titles, k=5):
    """
    Повертає top-k найбільш схожих питань для заданого текстового запиту.
    Використовує SBERT-ембедінги + sklearn.NearestNeighbors (cosine).
    """
    # Ембединг запиту
    q_emb = model.encode([query], convert_to_numpy=True, device=device).astype("float32")

    # Пошук сусідів
    distances, indices = nn.kneighbors(q_emb, n_neighbors=k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        sim = 1.0 - dist  # cosine similarity
        results.append({
            "title": titles.iloc[idx],
            "distance": float(dist),
            "similarity": float(sim)
        })
    return results

Потестуємо на кількох запитах

In [ ]:
queries = [
    "How to fix NullPointerException in Java?",
    "Train/test split for machine learning model",
    "How to center a div in CSS?"
]

for q in queries:
    print("\n" + "="*80)
    print("QUERY:", q)
    res = semantic_search(q, model, nn, unique_titles, k=5)
    for r in res:
        print(f"  sim={r['similarity']:.3f} | {r['title']}")

In [ ]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
from openai import OpenAI
client = OpenAI()

Функція для отримання ембедінга одного тексту

Для моделі text-embedding-3-large

In [ ]:
def get_embedding(text: str, model: str = "text-embedding-3-large"):
    text = text.replace("\n", " ")  # рекомендація OpenAI
    resp = client.embeddings.create(
        model=model,
        input=[text]
    )
    return resp.data[0].embedding

Ембедінги для всіх unique_titles

In [ ]:
# Наприклад, обмежимось 10 000 заголовків
max_titles = 10_000
titles_subset = unique_titles.iloc[:max_titles]

openai_embeddings = []

for i, text in enumerate(titles_subset):
    emb = get_embedding(text, model="text-embedding-3-large")
    openai_embeddings.append(emb)
    if (i + 1) % 1000 == 0:
        print(f"Processed {i+1} titles")

openai_embeddings = np.array(openai_embeddings, dtype="float32")
openai_embeddings.shape

In [ ]:
nn_openai = NearestNeighbors(
    n_neighbors=5,
    metric="cosine"
)
nn_openai.fit(openai_embeddings)

In [ ]:
def semantic_search_openai(query, client, nn, titles, model="text-embedding-3-large", k=5):
    # 1. Отримуємо ембедінг запиту
    q_emb = np.array(get_embedding(query, model=model), dtype="float32").reshape(1, -1)

    # 2. Шукаємо сусідів
    distances, indices = nn.kneighbors(q_emb, n_neighbors=k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        sim = 1.0 - dist
        results.append({
            "title": titles.iloc[idx],
            "distance": float(dist),
            "similarity": float(sim)
        })
    return results

In [ ]:
queries = [
    "How to fix NullPointerException in Java?",
    "Train/test split for machine learning model",
    "How to center a div in CSS?"
]

for q in queries:
    print("\n" + "="*80)
    print("QUERY:", q)
    res = semantic_search_openai(q, client, nn_openai, titles_subset, k=5)
    for r in res:
        print(f"  sim={r['similarity']:.3f} | {r['title']}")

## Notes

- Для OpenAI embeddings ключ можна передати через Colab Secrets (`OPENAI_API_KEY`) або через інтерактивний ввід (`getpass`).
- Для демо достатньо підмножини `unique_titles` (наприклад 20k), щоб контролювати вартість і час.
